In [48]:
import os
from glob import glob

import numpy as np
import torch
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
use_cuda = torch.cuda.is_available()

In [49]:
train_transforms = transforms.Compose([transforms.RandomRotation(60),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

image_path = 'data'

train_path = os.path.join(image_path, 'train')
val_path = os.path.join(image_path, 'valid')
test_path = os.path.join(image_path, 'test')

train_dataset = datasets.ImageFolder(train_path, train_transforms)
val_dataset = datasets.ImageFolder(val_path, train_transforms)
test_dataset = datasets.ImageFolder(test_path, test_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

loaders = {'train': train_loader, 'valid': val_loader, 'test': test_loader}

In [50]:
import torch.nn as nn
import torch.nn.functional as F

def calc_w_conv_out(conv, pool_stride = 1):
    return (((conv["W"] - conv["F"] + (2*conv["P"])) / conv["S"]) + 1) / pool_stride

conv1_w_in = 224
conv1 = {"W": conv1_w_in, "D": 3, "K": 4, "F": 7, "P": 0, "S": 7}
conv1_w_out = calc_w_conv_out(conv1)

conv2 = {"W": conv1_w_out, "D": conv1["K"], "K": 8, "F": 3, "P": 1, "S": 1}
conv2_w_out = calc_w_conv_out(conv2, 2)

conv3 = {"W": conv2_w_out, "D": conv2["K"], "K": 12, "F": 3, "P": 1, "S": 1}
conv3_w_out = calc_w_conv_out(conv3)

conv4 = {"W": conv3_w_out, "D": conv3["K"], "K": 16, "F": 3, "P": 1, "S": 1}
conv4_w_out = calc_w_conv_out(conv4, 4)

conv_features_out = conv4_w_out**2 * conv4["K"]


print(conv1_w_out, conv2_w_out, conv3_w_out, conv4_w_out, conv_features_out)

def make_nn_conv(conv):
    return nn.Conv2d(conv["D"], conv["K"], conv["F"], padding=conv["P"], stride=conv["S"])


# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        ## Layer 1
        self.conv1 = make_nn_conv(conv1)
        self.conv2 = make_nn_conv(conv2)
        ## Layer 2
        self.conv3 = make_nn_conv(conv3)
        self.conv4 = make_nn_conv(conv4)
        self.fc1 = nn.Linear(int(conv_features_out), 3)
                
    def forward(self, x):
        ## Define forward behavior
        batch_size = x.size()[0]

        # layer 1        
        x = F.dropout(F.relu(self.conv1(x)), 0.2)
        x = F.dropout(F.max_pool2d(F.relu(self.conv2(x)), 2, 2), 0.2)
        # layer 2
        x = F.dropout(F.relu(self.conv3(x)), 0.2)
        x = F.dropout(F.max_pool2d(F.relu(self.conv4(x)), 4, 4), 0.2)
        
        x = x.view(batch_size, -1)        
        x = self.fc1(x)
        
        return x


model = Net()

if use_cuda:
    model.cuda()

32.0 16.0 16.0 4.0 256.0


In [51]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [52]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            optimizer.zero_grad()
            
            output = model(data)
            
            loss = criterion(output, target)
            loss.backward()
            
            optimizer.step()
            
            train_loss += loss.item()*data.size(0)
        
        train_loss = train_loss/len(loaders['train'].sampler)

        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            
        valid_loss = valid_loss/len(loaders['valid'].sampler)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        if valid_loss < valid_loss_min:
            print(f'Saved model.pt, validation decresed: {valid_loss_min} => {valid_loss}')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), 'model.pt')
        
    return model

In [53]:
model = train(50, loaders, model, optimizer, 
                      criterion, use_cuda, 'model.pt')

Epoch: 1 	Training Loss: 0.907279 	Validation Loss: 1.076850
Saved model.pt, validation decresed: inf => 1.0768502831459046
Epoch: 2 	Training Loss: 0.828402 	Validation Loss: 1.073269
Saved model.pt, validation decresed: 1.0768502831459046 => 1.0732690556844076
Epoch: 3 	Training Loss: 0.826634 	Validation Loss: 1.083500
Epoch: 4 	Training Loss: 0.823217 	Validation Loss: 1.077728
Epoch: 5 	Training Loss: 0.818765 	Validation Loss: 1.063099
Saved model.pt, validation decresed: 1.0732690556844076 => 1.0630985196431477
Epoch: 6 	Training Loss: 0.829378 	Validation Loss: 1.071112
Epoch: 7 	Training Loss: 0.828556 	Validation Loss: 1.145380
Epoch: 8 	Training Loss: 0.823264 	Validation Loss: 1.166670
Epoch: 9 	Training Loss: 0.819240 	Validation Loss: 1.072838
Epoch: 10 	Training Loss: 0.817797 	Validation Loss: 1.050008
Saved model.pt, validation decresed: 1.0630985196431477 => 1.0500075284639994
Epoch: 11 	Training Loss: 0.816301 	Validation Loss: 1.036548
Saved model.pt, validation dec

In [54]:
model.load_state_dict(torch.load('model.pt'))
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

test(loaders, model, criterion, use_cuda)

Test Loss: 0.813358


Test Accuracy: 65% (393/600)


In [55]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [146]:
def make_pred(img):
    model.eval()
    if use_cuda:
        img.cuda()
    output = model(img.unsqueeze(0)).detach().numpy()

    return output

In [148]:
import pandas as pd

imgs = np.array(test_dataset.imgs)[:, 0]

pred_df = pd.DataFrame(data={'Id': imgs, 'task_1': 0, 'task_2': 0})
pred_df.head()


,Id,task_1,task_2
0,data/test/melanoma/ISIC_0012258.jpg,0,0
1,data/test/melanoma/ISIC_0012356.jpg,0,0
2,data/test/melanoma/ISIC_0012369.jpg,0,0
3,data/test/melanoma/ISIC_0012395.jpg,0,0
4,data/test/melanoma/ISIC_0012425.jpg,0,0


In [149]:
for i in range(len(imgs)):
    pred = make_pred(test_dataset.__getitem__(i)[0])
    pred_df.loc[i, 'task_1'] = pred[0][0]
    pred_df.loc[i, 'task_2'] = pred[0][2]
pred_df.head()

,Id,task_1,task_2
0,data/test/melanoma/ISIC_0012258.jpg,-0.657136,-0.041570
1,data/test/melanoma/ISIC_0012356.jpg,-0.072245,-0.325515
2,data/test/melanoma/ISIC_0012369.jpg,-0.296252,-0.318156
3,data/test/melanoma/ISIC_0012395.jpg,0.052060,-1.298566
4,data/test/melanoma/ISIC_0012425.jpg,-0.471274,-0.147245


In [150]:
pred_df.to_csv('predictions.csv', index=False)

In [153]:
#model.load_state_dict(torch.load('model.pt'))
import matplotlib.pyplot as plt
%matplotlib inline

!python get_results.py predictions.csv 0.4 

/Users/typekev/Library/Python/2.7/lib/python/site-packages/scipy/__init__.py:114: UserWarning: Numpy 1.8.2 or above is recommended for this version of scipy (detected version 1.8.0rc1)
  UserWarning)
Figure(640x480)
Category 1 Score: 0.553
Category 2 Score: 0.813
Category 3 Score: 0.683
Figure(640x480)
